In [1]:
import os

import PIL.Image
import tensorflow as tf
import librosa

import matplotlib
import matplotlib as plt

from matplotlib import figure

import numpy as np
from IPython.display import Audio

from tensorflow.keras.models import load_model
import pathlib

2022-11-17 09:37:52.084710: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-17 09:37:52.084749: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
%rm -rf ../content/testset

In [3]:
test_path='testset/'

genre_list=['bhojpuri_pop',
 'carnatic',
 'classic_bollywood',
 'desi_pop',
 'ghazal',
 'hindustani_classical',
 'indian_indie',
 'punjabi_hip_hop',
 'sufi',
 'tamil_pop']

In [4]:
file_name='../content/drive/MyDrive/moozyc/classic_bollywood/Kishore Kumar - Yeh Jo Mohabbat Hai.mp3'
file_name='../content/drive/MyDrive/moozyc/classic_bollywood/Aarti Mukherji - Do Naina Aur Ek Kahani.mp3'
file_name='../content/drive/MyDrive/moozyc/classic_bollywood/Lata Mangeshkar, Kishore Kumar - Is Mod Se Jate Hain - Duet.mp3'


#file_name="../content/drive/MyDrive/moozyc/carnatic/Nedunuri Krishnamurthy, R. Ramanathan, P.S. Devarajan - Dayarani - Mohanam.mp3"


# file_name='../content/drive/MyDrive/moozyc/indian_indie/Anumita Nadesan - Kabhi Kabhi.mp3'
# file_name='../content/drive/MyDrive/moozyc/indian_indie/Prateek Kuhad - All I Need.mp3'
# file_name='../content/drive/MyDrive/moozyc/indian_indie/When Chai Met Toast - Firefly.mp3'


file_name="../content/drive/MyDrive/moozyc/punjabi_hip_hop/AP Dhillon - Summer High.mp3"
# file_name="../content/drive/MyDrive/moozyc/punjabi_hip_hop/AP Dhillon, Shinda Kahlon, Gminxr - Arrogant.mp3"
# file_name="../content/drive/MyDrive/moozyc/punjabi_hip_hop/Sidhu Moose Wala - 295.mp3"

# file_name="../content/drive/MyDrive/moozyc/bhojpuri_pop/Ankush Raja - Chinese Love.mp3"
# file_name="../content/drive/MyDrive/moozyc/bhojpuri_pop/Indu Sonali - Hamare Pati Dev Ji.mp3"

# file_name="../content/drive/MyDrive/moozyc/tamil_pop/Teejay, Robert Sargunam - Hello Kekudha.mp3"
# file_name="../content/drive/MyDrive/moozyc/sufi/Satinder Sartaaj - Sajjan Raazi.mp3"
# file_name="../content/drive/MyDrive/moozyc/sufi/Shriram Iyer - Sufiya.mp3"

file_name="../FinalModel/recordings/demo.wav"

filenameparts=file_name.split('/')
song_name=filenameparts[-1]
song_name


'demo.wav'

In [5]:
%rm -rf testset

In [6]:
os.mkdir('testset')
dest_path='testset'

for genre in genre_list:
  os.mkdir(f'testset/{genre}')


In [7]:

def save_spectrogram(block,sr,genre,song_name,counter):
  matplotlib.use('Agg')
  S = librosa.feature.melspectrogram(y=block, sr=sr, n_mels=128,fmax=8000)
  plt.figure(figsize=(6, 3))
  librosa.display.specshow(librosa.power_to_db(S,ref=np.max),y_axis='mel', fmax=8000,x_axis='time')
  # plt.plot()
  song_name=song_name.replace('.mp3','')
  file_name=f'testset/{genre}/'+song_name+str(counter)+'.png'
  plt.savefig(file_name)
  plt.close()

In [8]:
audio, sr = librosa.load(file_name)

In [9]:
Audio(data=audio, rate=sr)

In [10]:
# Get number of samples for 6 seconds; replace 2 by any number
# Get number of samples for 6 seconds; replace 2 by any number
import loadsong

loadsong.make_spectrograms(audio,sr,song_name)

In [11]:
data_dir_test='testset'

In [12]:
#defining dataset dimensions
batch_size=64
image_height=100
image_width=200

#loading datasets

test_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir_test,
  seed=123,
  image_size=(image_height, image_width),
  batch_size=batch_size,
  shuffle="False"
  )

test_ds_cnn = tf.keras.utils.image_dataset_from_directory(
  data_dir_test,
  seed=123,
  image_size=(75, 150),
  batch_size=batch_size,
  shuffle="False"
  )

genre_list=test_ds.class_names

Found 4 files belonging to 10 classes.
Found 4 files belonging to 10 classes.


2022-11-17 09:37:59.114668: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-11-17 09:37:59.114737: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-11-17 09:37:59.114783: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (CoderCatA5Pop): /proc/driver/nvidia/version does not exist
2022-11-17 09:37:59.115244: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [13]:
import tensorflow
load_model=tensorflow.keras.models.load_model

In [14]:

def ensemble_models(model_genres:list,model_input,model_input_cnn):

    #loading all models
    models=[]
    models.append(load_model('./models/model1/'))
    models.append(load_model('./models/model2/'))
    models.append(load_model('./models/model3/'))
    models.append(load_model('./models/model4/'))

    #prediction
    model1=models[0].predict(model_input)
    model2=models[1].predict(model_input)
    model3=models[2].predict(model_input)
    model4=models[3].predict(model_input_cnn)

    avgEnsemble=(model1+model2+model3+model4)/len(models)

    avgLis=[]
    for i in avgEnsemble:
        avgLis.append(i.argmax())


    confi_lis=avgEnsemble[0]
    for i in avgEnsemble[1:]:
        confi_lis+=i
    confi_lis=confi_lis/(len(avgEnsemble))
    #print(confi_lis)
    summation=sum(confi_lis)

    for i in range(len(confi_lis)):
        acc=round(confi_lis[i]/summation,2)
        genr=model_genres[i]

        strin=f"{genr}\t\t\t\t{acc}%"

        print(strin.rjust(50,' '))


    #print(avgLis)
    most_confi=max(avgLis,key=avgLis.count)
    return model_genres[most_confi]


In [15]:
print(ensemble_models(genre_list,test_ds,test_ds_cnn))

                             bhojpuri_pop				0.15%
                                carnatic				-0.31%
                        classic_bollywood				0.11%
                                 desi_pop				0.26%
                                  ghazal				-0.02%
                     hindustani_classical				0.03%
                             indian_indie				-0.1%
                          punjabi_hip_hop				0.47%
                                     sufi				0.12%
                                tamil_pop				0.29%
punjabi_hip_hop


In [16]:
from testSong import testsong

print(testsong(file_name))

Found 4 files belonging to 10 classes.
Found 4 files belonging to 10 classes.
                             bhojpuri_pop				0.15%
                                carnatic				-0.31%
                        classic_bollywood				0.11%
                                 desi_pop				0.26%
                                  ghazal				-0.02%
                     hindustani_classical				0.03%
                             indian_indie				-0.1%
                          punjabi_hip_hop				0.47%
                                     sufi				0.12%
                                tamil_pop				0.29%
punjabi_hip_hop
punjabi_hip_hop


In [17]:
from sys import byteorder
from array import array
from struct import pack

import pyaudio
import wave

THRESHOLD = 500
CHUNK_SIZE = 1024
FORMAT = pyaudio.paInt16
RATE = 44100

def is_silent(snd_data):
    "Returns 'True' if below the 'silent' threshold"
    return max(snd_data) < THRESHOLD

def normalize(snd_data):
    "Average the volume out"
    MAXIMUM = 16384
    times = float(MAXIMUM)/max(abs(i) for i in snd_data)

    r = array('h')
    for i in snd_data:
        r.append(int(i*times))
    return r

def trim(snd_data):
    "Trim the blank spots at the start and end"
    def _trim(snd_data):
        snd_started = False
        r = array('h')

        for i in snd_data:
            if not snd_started and abs(i)>THRESHOLD:
                snd_started = True
                r.append(i)

            elif snd_started:
                r.append(i)
        return r

    # Trim to the left
    snd_data = _trim(snd_data)

    # Trim to the right
    snd_data.reverse()
    snd_data = _trim(snd_data)
    snd_data.reverse()
    return snd_data

def add_silence(snd_data, seconds):
    "Add silence to the start and end of 'snd_data' of length 'seconds' (float)"
    silence = [0] * int(seconds * RATE)
    r = array('h', silence)
    r.extend(snd_data)
    r.extend(silence)
    return r

def record():
    """
    Record a word or words from the microphone and 
    return the data as an array of signed shorts.

    Normalizes the audio, trims silence from the 
    start and end, and pads with 0.5 seconds of 
    blank sound to make sure VLC et al can play 
    it without getting chopped off.
    """
    p = pyaudio.PyAudio()
    stream = p.open(format=FORMAT, channels=1, rate=RATE,
        input=True, output=True,
        frames_per_buffer=CHUNK_SIZE)

    num_silent = 0
    snd_started = False

    r = array('h')

    while 1:
        # little endian, signed short
        snd_data = array('h', stream.read(CHUNK_SIZE))
        if byteorder == 'big':
            snd_data.byteswap()
        r.extend(snd_data)

        silent = is_silent(snd_data)

        if silent and snd_started:
            num_silent += 1
        elif not silent and not snd_started:
            snd_started = True

        if snd_started and num_silent > 30:
            break

    sample_width = p.get_sample_size(FORMAT)
    stream.stop_stream()
    stream.close()
    p.terminate()

    r = normalize(r)
    r = trim(r)
    r = add_silence(r, 0.5)
    return sample_width, r

def record_to_file(path):
    "Records from the microphone and outputs the resulting data to 'path'"
    sample_width, data = record()
    data = pack('<' + ('h'*len(data)), *data)

    wf = wave.open(path, 'wb')
    wf.setnchannels(1)
    wf.setsampwidth(sample_width)
    wf.setframerate(RATE)
    wf.writeframes(data)
    wf.close()

if __name__ == '__main__':
    print("please speak a word into the microphone")
    record_to_file('demo.wav')
    print("done - result written to demo.wav")

please speak a word into the microphone


ALSA lib pcm_dsnoop.c:566:(snd_pcm_dsnoop_open) unable to open slave
ALSA lib pcm_dmix.c:999:(snd_pcm_dmix_open) unable to open slave
ALSA lib pcm.c:2664:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.rear
ALSA lib pcm.c:2664:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.center_lfe
ALSA lib pcm.c:2664:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.side
ALSA lib pcm_oss.c:397:(_snd_pcm_oss_open) Cannot open device /dev/dsp
ALSA lib pcm_oss.c:397:(_snd_pcm_oss_open) Cannot open device /dev/dsp
ALSA lib confmisc.c:160:(snd_config_get_card) Invalid field card
ALSA lib pcm_usb_stream.c:482:(_snd_pcm_usb_stream_open) Invalid card 'card'
ALSA lib confmisc.c:160:(snd_config_get_card) Invalid field card
ALSA lib pcm_usb_stream.c:482:(_snd_pcm_usb_stream_open) Invalid card 'card'
ALSA lib pcm_dmix.c:999:(snd_pcm_dmix_open) unable to open slave


done - result written to demo.wav
